In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import numpy as np# Import library
import pandas as pd
from tqdm import tqdm
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
import csv

In [ ]:
user = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/대출 예측/데이터 전처리/user_spec_LE_진짜최종.csv', encoding = 'cp949')
user

In [ ]:
loan = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/대출 예측/데이터 전처리/loan_result_최종.csv', encoding = 'cp949')
loan

In [ ]:
loan_notnull = loan.loc[loan['is_applied'].notnull()]
loan_notnull['is_applied'].value_counts()

In [ ]:
loan_null = loan.loc[loan['is_applied'].isnull()]

In [ ]:
# user_spec 과 train inner 조인
loan_user = pd.merge(loan_notnull, user,on='application_id', how='inner')

In [ ]:
# 13519868 
loan_user

In [ ]:
loan_notnull = loan.loc[loan['is_applied'].notnull()]
loan_notnull['is_applied'].value_counts()

In [ ]:
loan_null = loan.loc[loan['is_applied'].isnull()]

In [ ]:
# user_spec 과 train inner 조인
loan_user = pd.merge(loan_notnull, user,on='application_id', how='inner')

In [ ]:
# 13519868 
loan_user

In [ ]:
loan_user['is_applied'].value_counts()

In [ ]:
loan_user.columns

In [ ]:
loan_user = loan_user[['application_id', 'user_id','bank_id', 'product_id', 'gender', 'Age',
       'loanapply_insert_time', 'loan_limit', 'loan_rate',  
       'credit_score', 'yearly_income', 'label_income_type',
       'label_employment_type', 'label_houseown_type', 'label_purpose',
       'desired_amount', 'personal_rehabilitation_yn','personal_rehabilitation_complete_yn',
        'existing_loan_cnt', 'existing_loan_amt',
       'work_day','is_applied']]

In [ ]:
loan_user = loan_user.drop(['loanapply_insert_time', 'application_id', 'user_id','bank_id'], axis = 1)

In [ ]:
loan_user.columns

In [ ]:
dataset = loan_user

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(dataset[['product_id', 'gender', 'Age', 'loan_limit', 'loan_rate',
       'credit_score', 'yearly_income', 'label_income_type',
       'label_employment_type', 'label_houseown_type', 'label_purpose',
       'desired_amount', 'personal_rehabilitation_yn',
       'personal_rehabilitation_complete_yn', 'existing_loan_cnt',
       'existing_loan_amt','work_day']], dataset['is_applied'])

train = pd.concat([X_train, y_train], axis = 1)
test = pd.concat([X_test, y_test], axis= 1)

In [ ]:
train

In [ ]:
!pip install pycaret

In [ ]:
!pip install numba==0.53

In [ ]:
!pip uninstall scikit-learn==1.0.2.
!pip install scikit-learn==0.23.2

In [ ]:
from pycaret.regression import *
from pycaret.classification import *

In [ ]:
train.columns

In [ ]:
clf = setup(data = train, target = 'is_applied', use_gpu=True)

In [ ]:
best_5 = compare_models(sort = 'AUC', n_select = 5)

In [ ]:
tuned_best_5 = [tune_model(i) for i in best_5]

In [ ]:
tuned_best_5

In [ ]:
blended = blend_models(estimator_list = best_5, method = 'soft')

In [ ]:
pred_holdout = predict_model(blended)

In [ ]:
final_model = finalize_model(blended)

In [ ]:
predictions = predict_model(final_model, data = test)